In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from lightgbm import LGBMClassifier

In [3]:
%%time
train = pd.read_parquet('/content/drive/MyDrive/AI/dataset/American Express/train_data.parquet')
label = pd.read_csv('/content/drive/MyDrive/AI/dataset/American Express/train_labels.csv')

CPU times: user 11.2 s, sys: 19.4 s, total: 30.6 s
Wall time: 44 s


In [4]:
label['target'].value_counts()

0    340085
1    118828
Name: target, dtype: int64

In [5]:
train = train.merge(label, on='customer_ID', how='inner')
display(train.head())
train_df = train[:10000]


del train
del label

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827,0


## 함수

In [6]:
def cat_col(data) :
    data['D_63'] = data['D_63'].astype('category')
    data['D_64'] = data['D_64'].astype('category')
  
    return data 

def report(model, X_test,Y_test):
    Y_pred = model.predict(X_test)
    return print(classification_report(Y_test, Y_pred))

In [7]:
train_df = train_df.dropna(thresh=5000,axis=1)
train_df = train_df.fillna(train_df.mean())

##날짜 변수 수치화 처리
train_df['S_2'] = pd.to_datetime(train_df['S_2'])
train_df['S_2'] = train_df['S_2'].dt.strftime("%Y%m%d").astype(int)

train_df = cat_col(train_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [8]:
train_df['target'].value_counts()

0    7447
1    2553
Name: target, dtype: int64

In [9]:
train_df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,20170309,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,0.005972,0.004345,0.001535,0.002427,0.003706,0.003818,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,20170407,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,0.004838,0.007495,0.004931,0.003954,0.003167,0.005032,0.009576,0.005492,0.009217,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,20170528,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,0.005497,0.009227,0.009123,0.003269,0.007329,0.000427,0.003429,0.006986,0.002603,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,20170613,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,0.008261,0.007206,0.002409,0.006117,0.004516,0.003200,0.008419,0.006527,0.009600,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,20170716,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,0.004848,0.006312,0.004462,0.003671,0.004946,0.008889,0.001670,0.008126,0.009827,0


In [10]:
features = train_df.columns

In [11]:
X = train_df.drop(['target','customer_ID'],axis=1)
y = train_df['target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,
                                                      shuffle=True, stratify = y, random_state=42)

In [12]:
X_train

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,...,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145
852,20180217,0.231466,0.031576,0.159524,0.039061,0.003199,0.125036,0.008685,0.254402,0.153766,...,1.003059,0.976519,0.174788,0.006530,0.002837,0.006192,0.008698,0.004985,0.001179,0.001441
4861,20170316,0.399364,0.003248,0.019104,0.247762,0.006152,0.175748,0.172948,0.084588,0.114594,...,0.000137,0.005720,0.009689,0.004138,0.005829,0.008329,0.008479,0.002640,0.008267,0.006755
2258,20171006,0.344771,2.508166,0.201076,0.073208,0.003249,1.398304,1.009423,0.500362,0.413611,...,1.008070,0.996171,0.360961,0.000595,0.007989,0.006481,0.003972,0.008455,0.005118,0.000512
8402,20170920,0.882457,0.008004,0.026390,1.007455,0.005536,0.133235,0.000089,0.010031,0.153766,...,0.006403,0.009719,0.002030,0.007670,0.002127,0.006606,0.009140,0.005317,0.008809,0.005909
4013,20170502,0.566359,0.652116,0.701700,0.037458,0.008536,0.246832,0.005704,0.854577,0.172558,...,1.007180,0.004640,0.006154,0.000647,1.000616,0.005643,0.864980,1.002202,0.003269,0.280966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8172,20170717,0.718471,0.295659,0.164615,0.254587,0.008355,0.131705,0.002022,0.112900,0.092332,...,0.006199,0.006241,0.005309,0.005688,0.002628,0.002003,0.007274,0.009551,0.005285,0.002534
4901,20170429,0.597679,0.002575,0.281269,0.058656,0.007070,0.104794,0.324054,0.576569,0.015232,...,0.003437,0.006596,0.007394,0.004987,0.006020,0.001119,0.006689,0.007503,0.003475,0.005071
4764,20170329,0.717628,0.009661,0.002900,0.810018,0.007627,0.227838,0.003824,0.008638,0.153766,...,0.000644,0.007033,0.005617,0.003030,0.005199,0.006711,0.003985,0.008875,0.001583,0.007182
9188,20170327,0.816076,0.005287,0.006177,0.813671,0.004797,0.227838,0.000578,0.002147,0.119601,...,0.000603,0.008508,0.008653,0.008285,0.008724,0.004920,0.005735,0.002404,0.000347,0.009978


In [13]:
params = {
          }

model = LGBMClassifier(**params)

model.fit(X_train, 
        y_train,
        eval_set= [(X_train, y_train), (X_valid, y_valid)], 
        eval_metric='auc', 
        verbose=0, 
        early_stopping_rounds=30
       )

LGBMClassifier()

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [15]:
a = np.array(y_valid)

b=a.reshape(-1,1)

In [16]:
b.shape

(2000, 1)

In [17]:
y_pred = model.predict(X_valid)

confusion_matrix(y_valid, y_pred)

report(model, X_valid, y_valid)

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1489
           1       0.96      0.93      0.95       511

    accuracy                           0.97      2000
   macro avg       0.97      0.96      0.97      2000
weighted avg       0.97      0.97      0.97      2000



In [18]:
test = pd.read_parquet('/content/drive/MyDrive/AI/dataset/American Express/test.parquet')

In [ ]:
test = test.dropna(thresh=5000,axis=1)
test = test.fillna(test.mean())

##날짜 변수 수치화 처리
test['S_2'] = pd.to_datetime(test['S_2'])
test['S_2'] = test['S_2'].dt.strftime("%Y%m%d").astype(int)

#test = cat_col(test)

In [ ]:
X_test = test.drop(['customer_ID'],axis=1)

In [ ]:
test["prediction"] = model.predict_proba(X_test)
test.head()

test.to_csv('/content/drive/MyDrive/AI/dataset/American Express/submission.csv', index=False)